# importing libararies and reading the data

In [122]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from sklearn.model_selection import train_test_split
print('Libraries imported.')

Libraries imported.


In [123]:
#adding the file from wikipedia first to local system excel and then reading it
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_9390d95550e445da8c20ad334fbf24ca = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='-AVhMeeS0poQAE7BvSmOq93V9r-ZFiIA8xrp2uOgihF8',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_9390d95550e445da8c20ad334fbf24ca.get_object(Bucket='datasciencecapstone-donotdelete-pr-knrirx6tn1wozq',Key='Neighbourhood.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_0 = pd.read_excel(body)
df_data_0.head()
df_data_0.shape

(180, 3)

# Task 1: notebook 1

In [124]:
#dropring Borough not assigned rows
df_data_1 = df_data_0.drop(df_data_0[df_data_0['Borough']=='Not assigned'].index)
df_data_1.shape

(103, 3)

In [125]:
#grouping data by postal code and adding neighborhood
df_data_2 = pd.DataFrame(df_data_1.groupby(by='Postal Code')['Neighbourhood'].sum())

In [126]:
#replacing original neighbourhood column with modified column 
df_data_1['Neighbourhood'].replace(df_data_2['Neighbourhood'],inplace=True)
print(df_data_1[df_data_1['Neighbourhood']=='Not assigned'], df_data_1.shape)

Empty DataFrame
Columns: [Postal Code, Borough, Neighbourhood]
Index: [] (103, 3)


In [127]:
# making not assigned neighbourhood value as same as it borough value
#this wont do any changes to data as we can show from previous cell that there are no values in df_data_1, Neighbourhood column as "Not assigned"
for i,Neighbour in enumerate(df_data_1['Neighbourhood']):
    if Neighbour=='Not assigned':
        df_data_1['Neighbourhood'] = df_data_1['Neighbourhood'].map({'Neighbour':df_data_1[i,'Borough']})
df_data_1.head(12)

,Postal Code,Borough,Neighbourhood
0,M1S,Scarborough,Agincourt
1,M8W,Etobicoke,"Alderwood, Long Branch"
2,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North"
3,M2K,North York,Bayview Village
4,M5M,North York,"Bedford Park, Lawrence Manor East"
5,M5E,Downtown Toronto,Berczy Park
6,M1N,Scarborough,"Birch Cliff, Cliffside West"
7,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place"
8,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
9,M6E,York,Caledonia-Fairbanks


In [128]:
df_data_1.shape

(103, 3)

# task 2: coordinates of each neighbourhood

In [129]:
#reding corrdinates csv file
body = client_9390d95550e445da8c20ad334fbf24ca.get_object(Bucket='datasciencecapstone-donotdelete-pr-knrirx6tn1wozq',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_3 = pd.read_csv(body)
df_data_3.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [130]:
df_data = pd.merge(df_data_1,df_data_3,on='Postal Code')

In [131]:
df_data.head(12)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1S,Scarborough,Agincourt,43.794200,-79.262029
1,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
2,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259
3,M2K,North York,Bayview Village,43.786947,-79.385975
4,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
7,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191
8,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
9,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512


# task3 : getting trending venues in neighbourhood of toronto and visulazing them

In [132]:
import folium

In [133]:
#defining foursqare credentials here, this part is removed because of confidentiality of credentials
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20200808'
LIMIT = 30

In [134]:
#selecting the data only relating to Borough Toronto
df_toronto = df_data[df_data['Borough'].str.contains('Toronto')]

In [135]:
#mapping all neighbourhoods of toronto on map using folium
#as all the coordinates are very close, using first neighbourhood coordinates to create the map
map_toronto = folium.Map(location=[df_toronto['Latitude'].values[0], df_toronto['Longitude'].values[0]], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [136]:
#getting nearby venues for each neighbourhood
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [137]:
#calling above funciton
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )
toronto_venues.head()

Berczy Park
Brockton, Parkdale Village, Exhibition Place
Business reply mail Processing Centre, South Central Letter Processing Plant Toronto
Central Bay Street
Christie
Church and Wellesley
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Commerce Court, Victoria Hotel
Davisville
Davisville North
Dufferin, Dovercourt Village
First Canadian Place, Underground city
Forest Hill North & West, Forest Hill Road Park
Garden District, Ryerson
Harbourfront East, Union Station, Toronto Islands
High Park, The Junction South
India Bazaar, The Beaches West
Kensington Market, Chinatown, Grange Park
Lawrence Park
Little Portugal, Trinity
Moore Park, Summerhill East
North Toronto West, Lawrence Park
Parkdale, Roncesvalles
Queen's Park, Ontario Provincial Government
Regent Park, Harbourfront
Richmond, Adelaide, King
Rosedale
Roselawn
Runnymede, Swansea
St. James Town
St. James Town, Cabbagetown
Stn A PO Boxes
Studio District
Summerhill West, Ra

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
1,Berczy Park,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
2,Berczy Park,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
3,Berczy Park,43.644771,-79.373306,Meridian Hall,43.646292,-79.376022,Concert Hall
4,Berczy Park,43.644771,-79.373306,Biff's Bistro,43.647085,-79.376342,French Restaurant


In [138]:
#checking how many venues are there for each neighbourhood
toronto_venues.groupby('Neighbourhood').count()['Venue Category']

Neighbourhood
Berczy Park                                                                                                   30
Brockton, Parkdale Village, Exhibition Place                                                                  24
Business reply mail Processing Centre, South Central Letter Processing Plant Toronto                          14
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport    15
Central Bay Street                                                                                            30
Christie                                                                                                      17
Church and Wellesley                                                                                          30
Commerce Court, Victoria Hotel                                                                                30
Davisville                                                                        

In [139]:
#checking how many unique venues are there
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 194 uniques categories.


# analysing each neighbourhood

In [140]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
toronto_onehot.set_index('Neighbourhood',inplace=True)
toronto_onehot.reset_index(inplace=True)
toronto_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts School,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Berczy Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Berczy Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,Berczy Park,0,0,0,0,

In [141]:
#grouping the data by neighbourhoods
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()

In [142]:
#getting top10 venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Beer Bar,Seafood Restaurant,Park,Italian Restaurant,Basketball Stadium,Bistro,Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Nightclub,Pet Store,Bar,Burrito Place,Restaurant,Climbing Gym,Yoga Studio
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Skate Park,Burrito Place,Farmers Market,Restaurant,Comic Shop,Pizza Place,Gym / Fitness Center,Garden,Garden Center
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Sculpture Garden,Plane,Airport Food Court
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Yoga Studio,Ramen Restaurant,Seafood Restaurant,Sandwich Place,Japanese Restaurant,Bubble Tea Shop,Poke Place


# clustering using Kmeans

In [143]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)


In [144]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() 

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Farmers Market,Beer Bar,Seafood Restaurant,Park,Italian Restaurant,Basketball Stadium,Bistro,Restaurant
7,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191,0,Café,Coffee Shop,Breakfast Spot,Nightclub,Pet Store,Bar,Burrito Place,Restaurant,Climbing Gym,Yoga Studio
8,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,0,Light Rail Station,Skate Park,Burrito Place,Farmers Market,Restaurant,Comic Shop,Pizza Place,Gym / Fitness Center,Garden,Garden Center
12,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Italian Restaurant,Café,Yoga Studio,Ramen Restaurant,Seafood Restaurant,Sandwich Place,Japanese Restaurant,Bubble Tea Shop,Poke Place
13,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Athletics & Sports,Candy Store,Restaurant,Diner,Nightclub,Italian Restaurant,Baby Store


In [145]:
# create map
map_clusters = folium.Map(location=[df_toronto['Latitude'].values[0], df_toronto['Longitude'].values[0]], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'],toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# analyzing data for each cluster

## cluster 1

In [146]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Farmers Market,Beer Bar,Seafood Restaurant,Park,Italian Restaurant,Basketball Stadium,Bistro,Restaurant
7,West Toronto,0,Café,Coffee Shop,Breakfast Spot,Nightclub,Pet Store,Bar,Burrito Place,Restaurant,Climbing Gym,Yoga Studio
8,East Toronto,0,Light Rail Station,Skate Park,Burrito Place,Farmers Market,Restaurant,Comic Shop,Pizza Place,Gym / Fitness Center,Garden,Garden Center
12,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Café,Yoga Studio,Ramen Restaurant,Seafood Restaurant,Sandwich Place,Japanese Restaurant,Bubble Tea Shop,Poke Place
13,Downtown Toronto,0,Grocery Store,Café,Park,Athletics & Sports,Candy Store,Restaurant,Diner,Nightclub,Italian Restaurant,Baby Store
14,Downtown Toronto,0,Gay Bar,Coffee Shop,Creperie,Japanese Restaurant,Dance Studio,Café,Hobby Shop,Juice Bar,Beer Bar,Indian Restaurant
17,Downtown Toronto,0,Airport Service,Airport Lounge,Airport,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Sculpture Garden,Plane,Airport Food Court
18,Downtown Toronto,0,Café,Gastropub,Restaurant,Japanese Restaurant,Coffee Shop,New American Restaurant,Seafood Restaurant,Sandwich Place,Pub,Museum
19,Central Toronto,0,Dessert Shop,Coffee Shop,Pizza Place,Café,Sandwich Place,Italian Restaurant,Gym,Sushi Restaurant,Farmers Market,Park
20,Central Toronto,0,Breakfast Spot,Park,Pizza Place,Gym,Sandwich Place,Department Store,Food & Drink Shop,Hotel,Creperie,Discount Store


## cluster 2

In [147]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,Central Toronto,1,Tennis Court,Yoga Studio,Creperie,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store


## cluster 3

In [148]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
71,Central Toronto,2,Garden,Yoga Studio,Cuban Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store


## cluster 4

In [149]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,Central Toronto,3,Jewelry Store,Trail,Sushi Restaurant,Mexican Restaurant,Colombian Restaurant,Comfort Food Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store


## cluster 5

In [150]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,Central Toronto,4,Park,Lawyer,Bus Line,Swim School,Cuban Restaurant,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store
70,Downtown Toronto,4,Park,Trail,Playground,Coworking Space,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega
